# Fetch Rewards Coding Exercise - Data Analyst

In [31]:
import pandas as pd
import numpy as np
from datetime import datetime

# Data Cleaning

### Receipts Dataframe Cleaning

In [92]:
receipts = pd.read_json('receipts.json', lines = True)

In [93]:
receipts.shape

(1119, 15)

In [94]:
# take a glance of the data
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


In [95]:
receipts['rewardsReceiptItemList'].describe()

count                                                   679
unique                                                  383
top       [{'description': 'flipbelt level terrain waist...
freq                                                     50
Name: rewardsReceiptItemList, dtype: object

#### There appear to be two major issues in this dataframe:
####    1. varables in some of the columns are not neat: the '_id' column, and the date columns - going to clean the
####        data and change the date data type
####    2. there are embeded dictionaries and lists in the column "rewardsReceiptItemList" - going to create another
####       dataframe to store this information and use receipt id ('_id') as the foreign key.
####        I didn't expand the column in the existing table because there are lots of Null value and will be too
####        redundent to include this column for every receipt id ('_id')

In [96]:
#create a function to convert the date data type
def date_convert(date):
    try:
        return (datetime.utcfromtimestamp(int(date['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return None

In [97]:
receipts['_id'] = receipts['_id'].apply(lambda x:x['$oid'])
receipts['createDate'] = receipts['createDate'].apply(lambda x:date_convert(x))
receipts['dateScanned'] = receipts['dateScanned'].apply(lambda x:date_convert(x))
receipts['finishedDate'] = receipts['finishedDate'].apply(lambda x:date_convert(x))
receipts['modifyDate'] = receipts['modifyDate'].apply(lambda x:date_convert(x))
receipts['pointsAwardedDate'] = receipts['pointsAwardedDate'].apply(lambda x:date_convert(x))
receipts['purchaseDate'] = receipts['purchaseDate'].apply(lambda x:date_convert(x))
receipts2 = receipts.drop(['rewardsReceiptItemList'], axis=1)
receipts_final = receipts2.fillna("Null")

In [98]:
receipts_final

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500,2021-01-03 00:00:00,5,FINISHED,26,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150,2021-01-02 15:24:43,2,FINISHED,11,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,Null,2021-01-03 15:25:42,Null,5,2021-01-03 00:00:00,1,REJECTED,10,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5,2021-01-03 00:00:00,4,FINISHED,28,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5,2021-01-02 15:25:06,2,FINISHED,1,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27,2021-03-01 10:22:27,Null,2021-03-01 10:22:28,Null,25,2020-08-17 00:00:00,2,REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,Null,Null,2021-03-01 15:42:41,2021-03-01 15:42:41,Null,2021-03-01 15:42:41,Null,Null,Null,Null,SUBMITTED,Null,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,Null,Null,2021-03-01 14:07:37,2021-03-01 14:07:37,Null,2021-03-01 14:07:37,Null,Null,Null,Null,SUBMITTED,Null,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,Null,2021-03-01 13:07:29,Null,25,2020-08-17 00:00:00,2,REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [99]:
#create a new table for 'rewardsReceiptItemList' column
rewardsReceiptItemList = receipts[['_id', 'rewardsReceiptItemList']]
rewardsReceiptItemList = rewardsReceiptItemList.explode('rewardsReceiptItemList')
rewardsReceiptItemList = rewardsReceiptItemList[rewardsReceiptItemList['rewardsReceiptItemList'].notna()]
rewardsReceiptItemList.reset_index(inplace=True)
rewardsReceiptItemList2 = pd.json_normalize(rewardsReceiptItemList['rewardsReceiptItemList'])
rewardsReceiptItemList_final1 = pd.concat([rewardsReceiptItemList.drop(['rewardsReceiptItemList'], axis=1),
                                         rewardsReceiptItemList2], axis=1)
rewardsReceiptItemList_final=rewardsReceiptItemList_final1.fillna('Null')

In [100]:
rewardsReceiptItemList_final

,index,_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5,...,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null
1,1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,Null,1,Null,1,...,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null
2,1,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1,...,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null
3,2,5ff1e1f10a720f052300057a,Null,Null,Null,Null,False,1,True,Null,...,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null
4,3,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4,...,Null,Null,Null,Null,Null,Null,Null,Null,Null,Null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,1113,603cc2bc0a720fde100003e9,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,Null,1,Null,1,...,Null,Null,Null,Null,Null,Null,Null,Null,11.99,Null
6937,1114,603cc0630a720fde100003e6,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,Null,0,Null,1,...,Null,Null,Null,Null,Null,Null,Null,Null,22.97,Null
6938,1114,603cc0630a720fde100003e6,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,Null,1,Null,1,...,Null,Null,Null,Null,Null,Null,Null,Null,11.99,Null
6939,1117,603ce7100a7217c72c000405,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,Null,0,Null,1,...,Null,Null,Null,Null,Null,Null,Null,Null,22.97,Null


### Users Dataframe Cleaning

In [42]:
users = pd.read_json('users.json', lines = True)
users.shape

(495, 7)

In [43]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


#### There appear to be the same issue: the '_id' column is not clean and the date data type needs to be converted

In [44]:
users['_id'] = users['_id'].apply(lambda x:x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x:date_convert(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x:date_convert(x))
users_final = users.fillna('Null')
users_final

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,Null,Null
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,Null,Null
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,Null,Null
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,Null,Null


### Brands Dataframe Cleaning

In [68]:
brands = pd.read_json('brands.json', lines = True)
brands.shape

(1167, 8)

In [69]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [70]:
#take a look at the 'cpg' column
brands['cpg'][0]

{'$id': {'$oid': '601ac114be37ce2ead437550'}, '$ref': 'Cogs'}

#### The '_id' column needs to be cleaned and the 'cpg' column needs to be split into two columns

In [71]:
brands['_id'] = brands['_id'].apply(lambda x:x['$oid'])
brands['cpg_id'] = brands['cpg'].apply(lambda x:x['$id']['$oid'])
brands['cpg_ref'] = brands['cpg'].apply(lambda x:x['$ref'])
brands = brands.drop(['cpg'], axis=1)
brands_final = brands.fillna('Null')

In [72]:
brands_final

,_id,barcode,category,categoryCode,name,topBrand,brandCode,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,test brand @1612366101024,0,Null,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,Starbucks,0,STARBUCKS,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,test brand @1612366146176,0,TEST BRANDCODE @1612366146176,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,test brand @1612366146051,0,TEST BRANDCODE @1612366146051,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,0,TEST BRANDCODE @1612366146827,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,511111116752,Baking,BAKING,test brand @1601644365844,Null,Null,5f77274dbe37ce6b592e90bf,Cogs
1163,5dc1fca91dda2c0ad7da64ae,511111706328,Breakfast & Cereal,Null,Dippin Dots® Cereal,Null,DIPPIN DOTS CEREAL,53e10d6368abd3c7065097cc,Cogs
1164,5f494c6e04db711dd8fe87e7,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,Null,TEST BRANDCODE @1598639215217,5332fa12e4b03c9a25efd1e7,Cogs
1165,5a021611e4b00efe02b02a57,511111400608,Grocery,Null,LIPTON TEA Leaves,0,LIPTON TEA Leaves,5332f5f6e4b03c9a25efd0b4,Cogs


In [53]:
#Write these four tables into csv file for SQL query
receipts_final.to_csv("receipts_final.csv", index=True, index_label="UniqueID")
rewardsReceiptItemList_final.to_csv("rewardsReceiptItemList_final.csv", index=True, index_label="UniqueID")
users_final.to_csv("users_final.csv", index=True, index_label="UniqueID")
brands_final.to_csv("brands_final.csv", index=True, index_label="UniqueID")

# Data Examination

In [55]:
# check users dataframe
users.isnull().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [58]:
users.shape

(495, 7)

In [59]:
users['_id'].duplicated().sum()

283

In [66]:
len(users._id.unique())

212

#### Quality issues in the users table: there are 283 dulicates ids out of 495 records, accounting for more than 50% of the table records; 
#### there are approxinately 10% missing values in some of the columns. 

In [73]:
# check the brands dataframe
brands.isnull().sum()

_id               0
barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
cpg_id            0
cpg_ref           0
dtype: int64

In [74]:
brands.shape

(1167, 9)

In [75]:
brands['_id'].duplicated().sum()

0

In [78]:
brands['name'].duplicated().sum()

11

In [80]:
len(brands.name.unique())

1156

In [79]:
brands['brandCode'].duplicated().sum()

269

In [81]:
len(brands.brandCode.unique())

898

#### Quality issues in the brands table: different '_id's may share the same brand name and/or barcode; 
#### the number of unique 'name' and 'barcode' doesn't match: one brandcode may have multiple brand names;
#### there are more than 50% of missing values in some of the columns. 

In [101]:
# check the receipts dataframe
receipts2.isnull().sum()

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [102]:
receipts2.shape

(1119, 14)

In [103]:
receipts[receipts['purchasedItemCount']==0]

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
15,5ff1e1e90a7214ada1000569,NaN,NaN,2021-01-03 15:25:29,2021-01-03 15:25:29,None,2021-01-03 15:25:29,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff1e1e9b6a9d73a3a9f10f6
81,5ff4ce3c0a720f05230005c4,NaN,NaN,2021-01-05 20:38:20,2021-01-05 20:38:20,None,2021-01-05 20:38:20,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff4ce3cc1e2d0121a9b2fba
141,5ff73be90a720f052300060a,NaN,NaN,2021-01-07 16:50:49,2021-01-07 16:50:49,None,2021-01-07 16:50:49,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff73be9eb7c7d31ca8a45bc
144,5ff794600a7214ada1000647,NaN,NaN,2021-01-07 05:08:16,2021-01-07 05:08:16,None,2021-01-07 17:08:17,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff7946004929111f6e90ceb
175,5ff8da570a720f05c5000015,NaN,NaN,2021-01-08 22:19:03,2021-01-08 22:19:03,None,2021-01-08 22:19:04,None,NaN,None,0.0,NaN,REJECTED,0.0,5ff8da28b3348b11c9337ac6
179,5ff8da7d0a720f05c500001c,NaN,NaN,2021-01-08 22:19:41,2021-01-08 22:19:41,None,2021-01-08 22:19:41,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,5ff8da7db3348b11c9337b6a
282,6000d4aa0a720f05f3000072,NaN,NaN,2021-01-14 23:32:58,2021-01-14 23:32:58,None,2021-01-14 23:32:58,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,6000d4aafb296c121a81b27e
396,6009eb000a7214ada2000003,250.0,"Receipt number 3 completed, bonus point schedu...",2021-01-21 20:58:40,2021-01-21 20:58:40,2021-01-21 20:59:06,2021-01-21 20:59:15,2021-01-21 20:59:06,250.0,2021-01-21 00:00:00,0.0,NaN,FINISHED,0.0,6009e60450b3311194385009
424,600aff160a720f053500000c,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-22 16:36:38,2021-01-22 16:36:38,2021-01-22 16:37:01,2021-01-22 16:37:13,2021-01-22 16:37:01,500.0,2021-01-22 00:00:00,0.0,NaN,FINISHED,0.0,600afb2a7d983a124e9aded0
544,60132b890a7214ad50000013,NaN,NaN,2021-01-28 21:24:25,2021-01-28 21:24:25,None,2021-01-28 21:24:25,None,NaN,None,0.0,"[{'needsFetchReview': True, 'needsFetchReviewR...",FLAGGED,0.0,60132b8873c60b3ca7f3bae8


#### Quality issues in the receipts table: for the receipts whose count of purchased items is zero still show points earned; 
#### there are more than 50% of missing values in some of the columns. 